# 03 - Interview Question Generation
`Author: Abdlazeez Jimoh`

In [1]:
import json
import importlib
from typing import Optional

from lib.agents import BaseAgent
from lib.configs import PALM_API_KEY
from lib.types import Question


class PalmQuestionGeneratorAgent(BaseAgent):
    def __init__(self):
        super().__init__()
        self.client = importlib.import_module("google.generativeai")
        self.client.configure(api_key=PALM_API_KEY)
        self.system_prompt = """You are a non-technical interviewer that interviews \
across the following categories:
- personal
- role-specific
- behavioural
- situational

You will be provided with a candidate's description.

Generate {n_questions} questions, ensuring that there is a question for each category \
and the questions should be based on the candidate's description.

* You answer strictly as a list of JSON objects. Don't include any other verbose texts, \
and don't include the markdown syntax anywhere.

JSON format:
[
    {{"question": "<personal_question>", "type": "personal"}},
    {{"question": "<role_specific_question>", "type": "role-specific"}},
    {{"question": "<behavioural_question>", "type": "behavioural"}},
    {{"question": "<situational_question>", "type": "situational"}},
    ...more questions to make up {n_questions} questions
]


===
Candidate Description:
{description}"""

    def __call__(self, description: str, n_questions: int = 4) -> Optional[list[Question]]:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            Optional[list[Question]]: A list of generated interview questions or None if an error occurs.
        """

        # Generate questions
        questions = self._generate(description, n_questions)

        return questions

    def run(self, description: str, n_questions: int = 4) -> Optional[list[Question]]:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            Optional[list[Question]]: A list of generated interview questions or None if an error occurs.
        """

        # Generate questions
        questions = self._generate(description, n_questions)

        return questions

    def _generate(self, description: str, n_questions: int) -> Optional[list[Question]]:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int): The number of questions to generate.

        Returns:
            Optional[list[Question]]: A list of generated interview questions or None if an error occurs.
        """

        try:
            # Ensure that there are at least 4 questions
            if n_questions < 4:
                n_questions = 4

            output = self.client.generate_text(
                model="models/text-bison-001",
                prompt=self.system_prompt.format(n_questions=n_questions, description=description),
                temperature=1,
                max_output_tokens=1024,
            )
            questions = json.loads(output.result)

            return questions
        except Exception as e:
            print(e)
            return None


In [2]:
from pprint import pprint


question_generator = PalmQuestionGeneratorAgent()
questions = question_generator.run(
    "a software engineer at a startup in San Francisco. I have 5 years of experience and I'm looking for a new job.",
    5,
)

pprint(questions)

/home/abdulazeezoj/Devspace/Work/omdena/local-chapter/india-chapter/chatbot-interview-prep/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'question': 'Why are you leaving your current job?', 'type': 'personal'},
 {'question': 'What are your salary expectations?', 'type': 'role-specific'},
 {'question': 'What is your greatest weakness?', 'type': 'behavioural'},
 {'question': 'Tell me about a time when you had to work under pressure',
  'type': 'situational'},
 {'question': 'What are your career goals?', 'type': 'personal'}]


In [3]:
questions = question_generator.run(
    "a data scientist from India. I have 3 years of experience. I've worked on a variety of projects, including a recommendation engine for a large e-commerce company. I've led a team of 5 data scientists and engineers.",
    8
)

pprint(questions)

[{'question': 'Why did you choose to become a data scientist?',
  'type': 'personal'},
 {'question': 'What are your greatest strengths and weaknesses as a data '
              'scientist?',
  'type': 'personal'},
 {'question': 'What are your goals for the future?', 'type': 'personal'},
 {'question': 'What is your experience with machine learning?',
  'type': 'role-specific'},
 {'question': 'What is your experience with big data technologies?',
  'type': 'role-specific'},
 {'question': 'What is your experience with working in a team?',
  'type': 'behavioural'},
 {'question': 'How do you handle stress?', 'type': 'behavioural'},
 {'question': 'Give me an example of a time when you had to make a difficult '
              'decision',
  'type': 'situational'},
 {'question': 'What are your thoughts on the ethics of using data science?',
  'type': 'situational'}]
